In [ ]:
import sys
import pandas as pd
sys.path.append("/Users/pvs262/Documents/rec-sys-dynamics/code")
from src.analysis.cluster import post_process, cluster

# change the run folder name
#fldr = "No_Explore_4"
fldr = "Explore_Threshold_2"

In [ ]:
def getLatents(folder):
    directory = "../simulation_runs/"+ fldr +"/"+ folder
    latents = []
    for i in range(100):
        df = pd.read_pickle(directory+'/L'+str(i)+'pkl.gzip', compression = 'gzip')
        latents.append(df)
        
    return latents

def getResults(folder):
    directory = "../simulation_runs/"+ fldr +"/"+ folder
    results = []
    for i in range(100):
        df = pd.read_pickle(directory+'/R'+str(i)+'pkl.gzip', compression = 'gzip')
        results.append(df)
        
    return results

def recalResults(folder):
    latents = getLatents(folder)
    results_list = []
    for i in range(len(latents)):
        cluster_obj = cluster(latents[i],0.3)
        results_list.append(cluster_obj.gmm(3, covariance_type='full', df='proba', svd = False))
    return results_list

def getUI(folder):
    directory = "../simulation_runs/"+ fldr +"/"+ folder
    UI = []
    for i in range(100):
        df = pd.read_pickle(directory+'/UI'+str(i)+'pkl.gzip', compression = 'gzip')   
        UI.append(df)
    return UI

### Create a dataset for animating iteration-wise clustering
Input: Results, Latents
Output: CSV of combined results, iteration# and first 3 latent features 

Step 1: save the new results for 2BCMU and 2BCLI runs (MF and cosin)

Step 2: collate results

Step 3: export as CSV

In [ ]:
'''
# STEP 1 (only needs to be run once)
sim = ['cosin_2BCMU_0_0_30_100','cosin_2BCLI_0_0_30_100','mf_2BCMU_0_0_30_100','mf_2BCLI_0_0_30_100']
sim = ['cosin_2BCMU_10_0_30_100','cosin_2BCLI_10_0_30_100','mf_2BCMU_10_0_30_100','mf_2BCLI_10_0_30_100']
for a in sim:
    latents = getLatents(a)
    results = recalResults(a)
    print("results recalculated. printing to folder.")

    for i in range(len(results)):
        results[i].to_pickle('../simulation_runs/'+fldr+"/"+a+'/R'+str(i)+'pkl.gzip', compression = 'gzip')
'''

In [ ]:
def collate2csv(folder):
    lats = getLatents(folder)
    res = getResults(folder)
    UI = getUI(folder)
    pp = post_process(lats, res, UI)
    res = pp.rename_cluster(1,99)
    
    final = pd.DataFrame()
    
    # change all latents to only have 3 features
    for i in range(len(lats)):
        # if # of latent features < 3, get latents from UI.
        if len(lats[i].columns) < 3:
            temp_obj = cluster(UI[i],3)
            temp_obj.svd()
            temp = temp_obj.data
        else:
            # get 3 most significant latent features
            temp = lats[i][lats[i].columns[0:3]]
        
        # swap columns for consistent axes
        if temp.iloc[0][2] > temp.iloc[0][1]:
            temp = temp.rename(columns={1: 2, 2: 1})
            print("column swapped in " + str(i))
        if temp.iloc[0][1] > temp.iloc[0][0]:
            temp = temp.rename(columns={1: 0, 0: 1})
            
        # update 3-column df in lats 
        lats[i] = temp.join(pd.DataFrame(res[i].cluster))
        lats[i]['iteration'] = i+1
        
        final = final.append(lats[i])
        
    final.to_csv("../simulation_runs/"+fldr+'/'+folder+".csv", index = False)
    
    return None        

In [ ]:
collate2csv('cosin_BNC_10_0_30_100')

In [ ]:

# No Explore
# folders = ['cosin_AN_0_0_30_100','cosin_BNC_0_0_30_100','cosin_1BCC_0_0_30_100','cosin_1BCMU_0_0_30_100','cosin_1BCLI_0_0_30_100','cosin_2BCC_0_0_30_100','cosin_2BCMU_0_0_30_100','cosin_2BCLI_0_0_30_100','mf_AN_0_0_30_100','mf_BNC_0_0_30_100','mf_1BCC_0_0_30_100','mf_1BCMU_0_0_30_100','mf_1BCLI_0_0_30_100','mf_2BCC_0_0_30_100','mf_2BCMU_0_0_30_100','mf_2BCLI_0_0_30_100']
#folders = ['item_based_AN_0_0_30_100','item_based_BNC_0_0_30_100','item_based_1BCC_0_0_30_100','item_based_1BCMU_0_0_30_100','item_based_1BCLI_0_0_30_100','item_based_2BCC_0_0_30_100','item_based_2BCMU_0_0_30_100','item_based_2BCLI_0_0_30_100']

# Explore
# folders = ['cosin_AN_10_0_30_100','cosin_BNC_10_0_30_100','cosin_1BCC_10_0_30_100','cosin_1BCMU_10_0_30_100','cosin_1BCLI_10_0_30_100','cosin_2BCC_10_0_30_100','cosin_2BCMU_10_0_30_100','cosin_2BCLI_10_0_30_100','mf_AN_10_0_30_100','mf_BNC_10_0_30_100','mf_1BCC_10_0_30_100','mf_1BCMU_10_0_30_100','mf_1BCLI_10_0_30_100','mf_2BCC_10_0_30_100','mf_2BCMU_10_0_30_100','mf_2BCLI_10_0_30_100']
#folders = ['item_based_AN_10_0_30_100','item_based_BNC_10_0_30_100','item_based_1BCC_10_0_30_100','item_based_1BCMU_10_0_30_100','item_based_1BCLI_10_0_30_100','item_based_2BCC_10_0_30_100','item_based_2BCMU_10_0_30_100','item_based_2BCLI_10_0_30_100']

for name in folders:
    collate2csv(name)
    


# Fixing Latent Feature Variations
E.g. cosin_BNC (explore) iteration 19 vs 20

In [ ]:
sim = 'cosin_BNC_10_0_30_100'
latents = getLatents(sim)
results = getResults(sim)
UI = getUI(sim)
cosin_BNC_10_0_30_100 = post_process(latents, results, UI)
cosin_BNC_10_0_30_100.rename_cluster(1,99)
cosin_BNC_10_0_30_100.plot_percent()

In [ ]:
latents[43][latents[43].columns[0:3]]

In [ ]:
latents[44][latents[44].columns[0:3]]

In [ ]:
test = latents[45][latents[45].columns[0:3]]
test

In [ ]:
test.rename(columns={1: 2, 2: 1})

In [ ]:
test.iloc[0][2]